<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/%EB%8F%99%EC%A0%81%EC%9E%90%EC%82%B0%EB%B0%B0%EB%B6%84_PAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 첫 설치 이후 설치 안함
!pip install yfinance

In [3]:
# 첫 설치 이후 설치 안함
!pip install pandas_market_calendars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.4/191.4 kB 6.6 MB/s eta 0:00:00


In [4]:
from pandas_datareader import data as pdr
import yfinance as yf

# yfinance 라이브러리 내장 기능을 활성화한다.
# yfinance의 데이터 가져오기 기능이 pandas_datareader 를 통해 사용할 수 있다.
yf.pdr_override()

import pandas as pd
import exchange_calendars as ecals
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import csv

# display에서 소수점 4자리까지 표현
pd.options.display.float_format = '{:.4f}'.format

# display시 DataFrame 모든 열 표시
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 11)

# ETF 가격 지정된 기간에서 가져와서 DataFrame에 담기
'''
def get_price_data(x, start_date, end_date):
  df_price = pd.DataFrame(columns=x)
  for ticker in x:
    df_price[ticker] = pdr.get_data_yahoo(ticker, start=start_date, end=end_date)['Adj Close']
  return df_price.dropna(axis=0)
'''

# 1년 전 개장일 구하기
us_market_code = "XNYS"
us_calendar = ecals.get_calendar(us_market_code)

# 각 날짜 변수에 저장
today = datetime.today().date()
today_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=0)).date())
month_01_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=30)).date())
month_03_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=91)).date())
month_06_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=182)).date())
month_12_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=365)).date())

# 전략에 사용할 ETF 명 리스트에 담기
PAA = ['SPY', 'QQQ', 'IWM', 'VGK', 'EWJ', 'EEM', 'IYR', 'GSG', 'GLD', 'HYG', 'LQD', 'TLT', 'IEF']

# ETF 가격 추출
# df_PAA = get_price_data(PAA, month_12_ago_workday, today).sort_index(ascending=False)
# [PAA] : PAA 순서대로 만들기
df_PAA = yf.download(PAA, start=month_12_ago_workday, end=today)['Adj Close'].sort_index(ascending=False)[PAA]
display(df_PAA)

[*********************100%%**********************]  13 of 13 completed


,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
Date,,,,,,,,,,,,,
2023-08-24,436.8900,361.2200,183.2200,59.5800,59.6600,38.8700,83.6400,21.1500,177.8500,74.3000,104.9000,94.9100,93.7200
2023-08-23,443.0300,369.1100,185.7900,60.4200,60.4500,39.0200,84.0000,21.1200,177.8900,74.7100,105.2600,95.5400,94.0700
2023-08-22,438.1500,363.3800,183.8300,59.9200,59.6600,38.4100,82.6700,21.1500,176.1000,74.1100,103.8000,93.2300,93.0600
2023-08-21,439.3400,363.9000,184.3800,60.1400,59.3700,38.4900,82.4000,21.2100,175.8100,74.0700,103.4800,92.5200,93.0200
2023-08-18,436.5000,358.1300,184.6500,59.8000,59.3200,38.4000,83.1300,21.2800,175.3300,74.0900,104.1000,93.7700,93.6000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-30,391.8118,298.8890,181.6512,50.1011,53.3420,38.3993,91.8588,22.7600,160.5100,70.7302,106.2737,109.6278,98.7900
2022-08-29,396.1608,302.2550,184.2291,50.4494,53.4014,38.8587,93.2177,23.4900,161.8600,71.2776,106.1390,109.3561,98.8193
2022-08-26,398.7977,305.2635,185.9411,50.3914,53.9464,39.1030,94.0524,23.0700,161.7600,71.5513,106.9281,110.2683,99.3846


In [5]:
# 12개월 이동평균 생성 및 저장
PAA_12mo_ma = df_PAA.rolling(window=252).mean().loc[month_12_ago_workday]  # 252 개의 영업일은 대략 12개월
df_PAA_12mo_ma = pd.DataFrame([PAA_12mo_ma], columns=PAA, index=['12mo_moving_average'])

# 가격 비율 계산 및 DataFrame 생성
df_PAA_12mo_ror = pd.DataFrame(index=['12개월 수익률'])
for symbol in PAA:
    price_today = df_PAA[symbol][0]
    price_12mo_ma = PAA_12mo_ma[symbol]
    ror = (price_today / price_12mo_ma - 1) * 100
    df_PAA_12mo_ror[symbol] = [ror]

df_PAA_12mo_ma = pd.concat([df_PAA_12mo_ma, df_PAA_12mo_ror])

# csv 경로 설정
output_path =  '/content/drive/MyDrive/DynamicAssetAllocation/'

# 오늘 날짜를 가져와서 파일명에 추가
today_date = datetime.now().strftime('%y%m%d')
filename = f'{today_date} PAA.xlsx'

# 오늘 날짜 string에 저장
str_today = today.strftime('### %Y-%m-%d ')

str_PAA_12mo_ma = str_today + "### 12개월 이동평균 및 12개월 수익률"
display(Markdown(str_PAA_12mo_ma))
display(df_PAA_12mo_ma)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'w') as f:
  f.write(f'{str_PAA_12mo_ma}\n')
  df_PAA_12mo_ma.to_csv(f, index=True)

# 0 이하 원소 개수 세기
count_negative_etf = (df_PAA_12mo_ror < 0).sum().sum().item()
str_count_negative_etf = "### 12개월 수익률 0 미만 ETF 개수: " + str(count_negative_etf)

display(Markdown(str_count_negative_etf))

# 안전자산 비율 계산
display(Markdown('### 안전자산 비율: ' + str(round((count_negative_etf) * (100/6), 1)) + '%'))

### 2023-08-25 ### 12개월 이동평균 및 12개월 수익률

,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
12mo_moving_average,403.9233,311.2490,180.5514,56.8099,56.5193,38.3979,85.1519,20.6944,172.7161,72.4175,104.9623,101.3168,95.9418
12개월 수익률,8.1616,16.0550,1.4781,4.8761,5.5568,1.2296,-1.7755,2.2017,2.9724,2.5995,-0.0593,-6.3235,-2.3157


### 12개월 수익률 0 미만 ETF 개수: 4

### 안전자산 비율: 66.7%

In [6]:
# 모멘텀스코어 계산할 오늘-1-3-6-12개월 날짜 저장
ago_name = ['today', 'month_01_ago', 'month_03_ago', 'month_06_ago', 'month_12_ago']
ago_date = [today_workday.date(), \
            month_01_ago_workday.date(), \
            month_03_ago_workday.date(), \
            month_06_ago_workday.date(), \
            month_12_ago_workday.date()]

# '개월 전/날짜' 형식으로 텍스트 저장
ago = [f'{a}/{d}' for a, d in zip(ago_name, ago_date)]

# ETF의 날짜별 데이터 담기
etf_data = {}

for i in range(len(PAA)):
  etf_data[PAA[i]] = [df_PAA.loc[today_workday, PAA[i]], \
                          df_PAA.loc[month_01_ago_workday, PAA[i]], \
                          df_PAA.loc[month_03_ago_workday, PAA[i]], \
                          df_PAA.loc[month_06_ago_workday, PAA[i]], \
                          df_PAA.loc[month_12_ago_workday, PAA[i]]]


## DataFrame 만들기: 1-3-6-12 개월 etf 가격
df_etf_price = pd.DataFrame(etf_data, columns=PAA, index=ago)
str_etf_price = str_today + "### ETF 가격 현재-1-3-6-12 개월"

display(Markdown(str_etf_price))
display(df_etf_price)

# csv에 제목 및 DataFrame 추가. 드라이브 마운트 후 해야함
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_price}\n')
  df_etf_price.to_csv(f, index=True)

### 2023-08-25 ### ETF 가격 현재-1-3-6-12 개월

,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
today/2023-08-24,436.8900,361.2200,183.2200,59.5800,59.6600,38.8700,83.6400,21.1500,177.8500,74.3000,104.9000,94.9100,93.7200
month_01_ago/2023-07-25,455.4400,378.7800,194.9200,63.2200,62.5800,41.2100,89.2300,21.3700,182.4100,74.8335,107.3016,100.8919,95.7873
month_03_ago/2023-05-25,413.1154,339.2546,173.6890,59.7834,59.6394,38.0221,80.1417,19.1500,180.2000,72.4812,104.9757,99.4699,96.0459
month_06_ago/2023-02-23,397.6595,295.9552,188.0634,59.1883,56.0740,38.8754,87.5755,20.1700,169.5700,72.4263,104.5472,100.7140,94.5921
month_12_ago/2022-08-24,407.0234,312.7800,189.3159,51.4462,54.6698,38.8489,94.9551,23.2900,163.2500,72.0704,106.4758,107.9392,98.8680


In [7]:
etf_momentum_score = []
etf_ror_data = []

## DataFrame 만들기: 수익률
for etf_name in PAA:
  etf_today_price = df_etf_price.loc[ago[0], etf_name]
  etf_momentum_score.append((12 * (etf_today_price/df_etf_price.loc[ago[1], etf_name] - 1) * 100) + \
                            (4 * (etf_today_price/df_etf_price.loc[ago[2], etf_name] - 1) * 100) + \
                            (2 * (etf_today_price/df_etf_price.loc[ago[3], etf_name] - 1) * 100) + \
                            (1 * (etf_today_price/df_etf_price.loc[ago[4], etf_name] - 1) * 100))

  etf_ror_data.append([(etf_today_price/df_etf_price.loc[ago[1], etf_name] - 1) * 100, \
                       (etf_today_price/df_etf_price.loc[ago[2], etf_name] - 1) * 100, \
                       (etf_today_price/df_etf_price.loc[ago[3], etf_name] - 1) * 100, \
                       (etf_today_price/df_etf_price.loc[ago[4], etf_name] - 1) * 100])

df_etf_ror = pd.DataFrame(etf_ror_data, columns=ago[1:], index=PAA).transpose()

str_etf_ror = str_today + "### ETF 1-3-6-12 개월 수익률"
display(Markdown(str_etf_ror))
display(df_etf_ror)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror}\n')
  df_etf_ror.to_csv(f, index=True)
  # df_etf_ror.to_csv(filename, mode='a', header=True, index=True)

### 2023-08-25 ### ETF 1-3-6-12 개월 수익률

,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
month_01_ago/2023-07-25,-4.0730,-4.6359,-6.0025,-5.7577,-4.6660,-5.6782,-6.2647,-1.0295,-2.4999,-0.7129,-2.2382,-5.9290,-2.1582
month_03_ago/2023-05-25,5.7549,6.4746,5.4874,-0.3402,0.0345,2.2299,4.3652,10.4439,-1.3041,2.5094,-0.0721,-4.5842,-2.4217
month_06_ago/2023-02-23,9.8653,22.0522,-2.5754,0.6618,6.3951,-0.0140,-4.4939,4.8587,4.8829,2.5870,0.3374,-5.7629,-0.9220
month_12_ago/2022-08-24,7.3378,15.4869,-3.2200,15.8102,9.1278,0.0543,-11.9163,-9.1885,8.9433,3.0936,-1.4800,-12.0708,-5.2069


In [8]:
## DataFrame 만들기: 수익률 가중치
df_etf_ror_weight = df_etf_ror
df_etf_ror_weight = df_etf_ror_weight.apply(lambda row: row * 12 if row.name == ago[1] else row, axis=1)
df_etf_ror_weight = df_etf_ror_weight.apply(lambda row: row * 4 if row.name == ago[2] else row, axis=1)
df_etf_ror_weight = df_etf_ror_weight.apply(lambda row: row * 2 if row.name == ago[3] else row, axis=1)

str_etf_ror_weight = str_today + "### ETF 1-3-6-12 개월 가중치 적용"
display(Markdown(str_etf_ror_weight))
display(df_etf_ror_weight)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror_weight}\n')
  df_etf_ror_weight.to_csv(f, index=True)

### 2023-08-25 ### ETF 1-3-6-12 개월 가중치 적용

,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
month_01_ago/2023-07-25,-48.8758,-55.6312,-72.0295,-69.0920,-55.9924,-68.1388,-75.1766,-12.3538,-29.9983,-8.5545,-26.8583,-71.1484,-25.8982
month_03_ago/2023-05-25,23.0198,25.8984,21.9496,-1.3608,0.1379,8.9198,17.4608,41.7755,-5.2164,10.0375,-0.2885,-18.3369,-9.6866
month_06_ago/2023-02-23,19.7307,44.1045,-5.1508,1.3236,12.7903,-0.0280,-8.9877,9.7174,9.7659,5.1741,0.6748,-11.5258,-1.8440
month_12_ago/2022-08-24,7.3378,15.4869,-3.2200,15.8102,9.1278,0.0543,-11.9163,-9.1885,8.9433,3.0936,-1.4800,-12.0708,-5.2069


In [9]:
## DataFrame 만들기: 모멘텀
df_etf_momentum_score = pd.DataFrame(etf_momentum_score, columns=['momentum_score'], index=PAA).transpose()

str_etf_momentum_score = str_today + "### ETF 별 12개월 Momentum Score"
display(Markdown(str_etf_momentum_score))
display(df_etf_momentum_score)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_momentum_score}\n')
  df_etf_momentum_score.to_csv(f, index=True)

### 2023-08-25 ### ETF 별 12개월 Momentum Score

,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
momentum_score,1.2125,29.8586,-58.4507,-53.3190,-33.9363,-59.1927,-78.6198,29.9505,-16.5055,9.7507,-27.9519,-113.0819,-42.6358


In [13]:
# 12개월 수익률 (-)인 ETF 개수 및 16.7%씩 투자할 자산 개수와 종목
count_invest_etf = (lambda count_negative_etf: 6 - count_negative_etf if count_negative_etf < 7 else 0)(count_negative_etf)
str_count_invest_etf = "## 16.7% 씩 투자할 자산 개수: " + str(count_invest_etf)

copy_etf_momentum_score = etf_momentum_score.copy()
recommended_etfs = []
for i in range(count_invest_etf):
  recommended_etfs.append(PAA[copy_etf_momentum_score.index(max(copy_etf_momentum_score))])
  copy_etf_momentum_score[copy_etf_momentum_score.index(max(copy_etf_momentum_score))] = -99999  # 선택된 ETF 점수를 낮은 값으로 변경

str_etf_PAA = "## 및 추천 종목: " + ' '.join(recommended_etfs)
str_etf_ratio = "## 투자 비율 - 현금대체재 | 보호막 자산: " + str(round(count_negative_etf * 1/6 * 100, 1)) + "% | " + str(round(count_invest_etf * 1/6 * 100, 1)) + "%"

display(Markdown(str_today[1:]))
display(Markdown(str_count_negative_etf[1:]))
display(Markdown(str_count_invest_etf))
display(Markdown(str_etf_PAA))
display(Markdown(str_etf_ratio))

# csv에 자산 투자 조언 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_today[1:]}\n')
  f.write(f'{str_count_negative_etf[1:]}\n')
  f.write(f'{str_count_invest_etf}\n')
  f.write(f'{str_etf_PAA}')
  f.write(f'{str_etf_ratio}')

## 2023-08-25 

## 12개월 수익률 0 미만 ETF 개수: 4

## 16.7% 씩 투자할 자산 개수: 2

## 및 추천 종목: GSG QQQ

## 투자 비율 - 현금대체재 | 보호막 자산: 66.7% | 33.3%